In [ ]:
import nglview
import ipywidgets

#pdb_code = "4i23"
pdb_code = "2rgp"
drugbank_code = "DB11963"
ligand_remove = "HYZ"
compare_id = "4hjo"
api_id = "pdbe"
pockets_dir = "pockets"

In [ ]:
from biobb_io.api.pdb import pdb

In [ ]:
download_pdb = "download.pdb"
prop = {
  "pdb_code": pdb_code,
  "filter": ["ATOM", "HETATM"],
  "api_id": api_id
}

pdb(output_pdb_path=download_pdb,
    properties=prop)

In [ ]:
view = nglview.show_structure_file(download_pdb, default=True)
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_structure_utils.utils.remove_ligand import remove_ligand

In [ ]:
pdb_no_ligand = "pdb_no_ligand.pdb"
prop = {
    "ligand": ligand_remove
}

remove_ligand(input_structure_path=download_pdb,
             output_structure_path = pdb_no_ligand,
            properties=prop)

In [ ]:
view = nglview.show_structure_file(pdb_no_ligand, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                       colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_structure_utils.utils.extract_chain import extract_chain

In [ ]:
pdb_single_chain = "extract_chain.pdb"
prop = {
    "chains": ["A"]
}

extract_chain(input_structure_path=pdb_no_ligand,
             output_structure_path = pdb_single_chain,
            properties=prop)

In [ ]:
view = nglview.show_structure_file(pdb_single_chain, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_vs.fpocket.fpocket import fpocket

In [ ]:
fpocket_all_pockets = "fpocket_all_pockets.zip"
fpocket_summary = "fpocket_summary.json"
prop = {
    "min_radius": 3,
    "max_radius": 6,
    "num_spheres": 35
}

fpocket(input_pdb_path=pdb_single_chain,
        output_pockets_zip = fpocket_all_pockets,
        output_summary=fpocket_summary,
        properties=prop)

In [ ]:
import json

with open(fpocket_summary, 'r') as json_file:
    data = json.load(json_file)
    print(json.dumps(data, indent=4))

In [ ]:
from biobb_vs.fpocket.fpocket_filter import fpocket_filter

In [ ]:
fpocket_filter_pockets = "fpocket_filter_pockets.zip"
prop = {
    "druggability_score": [0.04, 1],
    "volume": [100, 600.2]
}

fpocket_filter(input_pockets_zip=fpocket_all_pockets,
                input_summary = fpocket_summary,
                output_filter_pockets_zip=fpocket_filter_pockets,
                properties=prop)

In [ ]:
import os
from pathlib import Path, PurePath
import zipfile

if not Path(pockets_dir).exists(): os.mkdir(pockets_dir)
#fpocket_filter_pockets
with zipfile.ZipFile(fpocket_all_pockets, 'r') as zip_ref:
    zip_ref.extractall(pockets_dir)

path_pockets = [str(i) for i in Path(pockets_dir).iterdir()]
path_pockets_pdb = [str(i) for i in Path(pockets_dir).iterdir() if PurePath(i).suffix == '.pdb']
path_pockets_pqr = [str(i) for i in Path(pockets_dir).iterdir() if PurePath(i).suffix == '.pqr']

In [ ]:
import re
import random

# random colors for cavities
r = lambda: random.randint(0,255)

# load structure
view = nglview.NGLWidget()
c = view.add_component(pdb_single_chain)

# load cavities (d) and pockets (p) and create pocketNames list
c = {}
p = {}
pocketNames = []
for pock in path_pockets:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    suff = g[0][1]
    if not [item for item in pocketNames if ('pocket' + i) in item]: pocketNames.append(('pocket' + i, int(i)))

    if suff == 'pdb':
        c[i] = view.add_component(pock)
        c[i].clear()
    else:
        p[i] = view.add_component(pock)
        p[i].clear()

# sort pocket names
pocketNames.sort(key=lambda tup: tup[1])
        
# representation for cavities
for pock in path_pockets_pdb:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    c[i].add_surface(color='#%02X%02X%02X' % (r(),r(),r()),  
                     radius='1.5',
                     lowResolution= True,
                     # 0: low resolution 
                     smooth=1,
                     useWorker= True,
                     wrap= True)
    
# representation for pockets
for pock in path_pockets_pqr:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    p[i].add_surface( component=i, color='skyblue', surfaceType= 'av', contour=True )

view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

# show pocket labels
code = """
var stage = this.stage;
var view = this.stage.viewer;
var clist_len = stage.compList.length;
var i = 0;
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined && stage.compList[i].object.name.indexOf('pqr') != -1) {        

        var elm = document.createElement("div");
        elm.innerText = 'pocket' + stage.compList[i].object.name.match(/\d+/g)
        elm.style.color = "black";
        elm.style.background = "rgba(201, 149, 6, .8)";
        elm.style.padding = "8px";
        
        stage.compList[i].addAnnotation(stage.compList[i].structure.center, elm)
    }
}
"""

view._execute_js_code(code)

view

In [ ]:
mdsel = ipywidgets.Dropdown(
    options=pocketNames,
    description='Sel. pocket:',
    disabled=False,
)
display(mdsel)

In [ ]:
from biobb_vs.fpocket.fpocket_select import fpocket_select

In [ ]:
fpocket_cavity = "fpocket_cavity.pdb"
fpocket_pocket = "fpocket_pocket.pqr"
prop = {
    "pocket": mdsel.value
}

fpocket_select(input_pockets_zip=fpocket_all_pockets,
#fpocket_select(input_pockets_zip=fpocket_filter_pockets,
                output_pocket_pdb = fpocket_cavity,
                output_pocket_pqr=fpocket_pocket,
                properties=prop)

In [ ]:
from biobb_vs.utils.box import box

In [ ]:
output_box = "box.pdb"
prop = {
    "offset": 10,
    "box_coordinates": True
}

box(input_pdb_path = fpocket_pocket,
            output_pdb_path = output_box,
            properties=prop)

In [ ]:
#view = nglview.show_structure_file(box, default=False)
view = nglview.NGLWidget()
#s = view.add_component(pdb_single_chain)
s = view.add_component(download_pdb)
b = view.add_component(output_box)
p = view.add_component(fpocket_pocket)
p.clear()

atomPair = [
    [ "9999:Z.ZN1", "9999:Z.ZN2" ],
    [ "9999:Z.ZN2", "9999:Z.ZN4" ],
    [ "9999:Z.ZN4", "9999:Z.ZN3" ],
    [ "9999:Z.ZN3", "9999:Z.ZN1" ],
    
    [ "9999:Z.ZN5", "9999:Z.ZN6" ],
    [ "9999:Z.ZN6", "9999:Z.ZN8" ],
    [ "9999:Z.ZN8", "9999:Z.ZN7" ],
    [ "9999:Z.ZN7", "9999:Z.ZN5" ],
    
    [ "9999:Z.ZN1", "9999:Z.ZN5" ],
    [ "9999:Z.ZN2", "9999:Z.ZN6" ],
    [ "9999:Z.ZN3", "9999:Z.ZN7" ],
    [ "9999:Z.ZN4", "9999:Z.ZN8" ]
]

# structure
s.add_representation(repr_type='cartoon', 
                        selection='not het',
                        color='#cccccc',
                       opacity=.2)
# ligands box
b.add_representation(repr_type='ball+stick',
                     selection='9999',
                     color='pink', 
                     aspectRatio = 8)
# lines box
b.add_representation(repr_type='distance', 
                     atomPair= atomPair,
                     labelVisible=False,
                     color= 'black')

# pocket
p.add_surface(component=mdsel.value, 
              color='skyblue', 
              surfaceType= 'av', 
              lowResolution= True,
              # 0: low resolution 
              smooth=1,
              contour=True,
              opacity=0.4,
              useWorker= True,
              wrap= True )


view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

In [ ]:
from biobb_io.api.ideal_sdf import ideal_sdf

In [ ]:
sdf_ideal = "ideal.sdf"
prop = {
  "ligand_code": ligand_remove
}

ideal_sdf(output_sdf_path=sdf_ideal,
    properties=prop)

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
ligand = "ligand.pdb"
prop = {
    "input_format": "sdf",
    "output_format": "pdb",
    "obabel_path": "/anaconda3/bin/obabel"
}

babel_convert(input_path = sdf_ideal,
            output_path = ligand,
            properties=prop)

In [ ]:
# ONLY IN CASE NOT IDEAL SDF DOWNLOADED
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

In [ ]:
# ONLY IN CASE NOT IDEAL SDF DOWNLOADED
ligand = "ligand.pdb"
prop = {
    "heteroatoms": [{ "name": ligand_remove }]
}

extract_heteroatoms(input_structure_path = download_pdb,
            output_heteroatom_path = ligand,
            properties=prop)

In [ ]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

In [ ]:
ligand_H = "ligand_H.pdb"
prop = {
    
}

reduce_add_hydrogens(input_path=ligand,
                    output_path=ligand_H,
                    properties=prop)

In [ ]:
from ipywidgets import HBox

In [ ]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(ligand_H)

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
prep_ligand = "prep_ligand.pdbqt"
prop = {
    "input_format": "pdb",
    "output_format": "pdbqt",
    "obabel_path": "/anaconda3/bin/obabel"
}

babel_convert(input_path = ligand_H,
            output_path = prep_ligand,
            properties=prop)

In [ ]:
from biobb_structure_utils.utils.str_check_add_hydrogens import str_check_add_hydrogens

In [ ]:
prep_receptor = "prep_receptor.pdbqt"
prop = {
    "charges": True,
    "mode": "auto"
}

str_check_add_hydrogens(input_structure_path = pdb_single_chain,
            output_structure_path = prep_receptor,
            properties=prop)

In [ ]:
from biobb_vs.vina.autodock_vina import autodock_vina

In [ ]:
output_vina_pdbqt = "output_vina.pdbqt"
output_vina_log = "output_vina.log"

autodock_vina(input_ligand_pdbqt_path = prep_ligand,
             input_receptor_pdbqt_path = prep_receptor,
             input_box_path = output_box,
            output_pdbqt_path = output_vina_pdbqt,
             output_log_path = output_vina_log)

In [ ]:
models = 'all'
#models = '/0 or /1 or /4'

v0 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v0.add_representation(repr_type='licorice', 
                        selection=models,
                       colorScheme= 'partialCharge')
v0.center()
v1 = nglview.show_structure_file(output_vina_pdbqt, default=False)
v1.add_representation(repr_type='ball+stick', 
                        selection=models)
v1.center()

v0._set_size('500px', '')
v1._set_size('500px', '')

def on_change(change):
    v1._set_camera_orientation(change['new'])
    
v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from Bio.PDB import PDBParser
parser = PDBParser(QUIET = True)
structure = parser.get_structure("protein", output_vina_pdbqt)
models = []
for i, m in enumerate(structure):
    models.append(('model' + str(i), i))
    
mdsel = ipywidgets.Dropdown(
    options=models,
    description='Sel. model:',
    disabled=False,
)
display(mdsel)

In [ ]:
from biobb_vs.utils.extract_model_pdbqt import extract_model_pdbqt

In [ ]:
output_pdbqt_model = "output_model.pdbqt"
prop = {
    "model": mdsel.value + 1
}

extract_model_pdbqt(input_pdbqt_path = output_vina_pdbqt,
             output_pdbqt_path = output_pdbqt_model,
            properties=prop)

In [ ]:
v0 = nglview.show_structure_file(ligand)
v1 = nglview.show_structure_file(output_pdbqt_model)

v0._set_size('500px', '')
v1._set_size('500px', '')

#def on_change(change):
#    v1._set_camera_orientation(change['new'])
    
#v0.observe(on_change, ['_camera_orientation'])

HBox([v0, v1])

In [ ]:
from biobb_chemistry.babelm.babel_convert import babel_convert

In [ ]:
output_pdb_model = "output_model.pdb"
prop = {
    "input_format": "pdbqt",
    "output_format": "pdb",
    "obabel_path": "/anaconda3/bin/obabel"
}

babel_convert(input_path = output_pdbqt_model,
             output_path = output_pdb_model,
            properties=prop)

In [ ]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

In [ ]:
output_structure = "output_structure.pdb"

cat_pdb(input_structure1 = pdb_single_chain,
             input_structure2 = output_pdb_model,
             output_structure_path = output_structure)

In [ ]:
# download 4hjo for comparing purposes
compare_structure = compare_id + ".pdb"
prop = {
  "pdb_code": compare_id,
  "filter": ["ATOM", "HETATM"],
  "api_id": api_id
}
compare_structure
pdb(output_pdb_path=compare_structure,
    properties=prop)

In [ ]:
# load reference and output
view = nglview.NGLWidget()
#v1 = view.add_component(compare_structure)
v1 = view.add_component(download_pdb)
v2 = view.add_component(output_structure)
v1.clear()
v1.add_representation(repr_type='licorice', 
                        selection='HYZ',
                      #selection='AQ4',
                     radius=0.5)

view._remote_call('setSize', target='Widget', args=['','600px'])
view

something = 'kk'

# align reference and output
code = """
console.log(%s)
var stage = this.stage;
var clist_len = stage.compList.length;
var i = 0;
var s = [];
//console.log(clist_len)
//console.log(stage.compList[0])
//console.log(stage.compList)
for(i = 0; i <= clist_len; i++){
    //console.log(stage.compList[i])
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined) {        
       s.push(stage.compList[i])
    }
}

NGL.superpose(s[0].structure, s[1].structure, true)
s[ 0 ].updateRepresentations({ position: true })
s[ 0 ].autoView()
""" % something

view._execute_js_code(code)
view

In [ ]:
#v0 = nglview.show_structure_file(compare_structure)
#v0 = nglview.show_structure_file(download_pdb)

#v1 = nglview.show_structure_file(output_structure)

#v0._set_size('500px', '500px')
#v1._set_size('500px', '500px')

#def on_change(change):
#    v1._set_camera_orientation(change['new'])
    
#v0.observe(on_change, ['_camera_orientation'])

#HBox([v0, v1])